In [ ]:
from re import I
import librosa,sys
import matplotlib.pyplot as plt
import numpy as np
from librosa import display
import scipy
import math
import os
from random import *
from skimage import io
from skimage import color
from skimage.util import img_as_ubyte

def fft_plot(audio, sampling_rate):
  n=len(audio)
  T= 1/sampling_rate
  yf= scipy.fft.fft(audio)
  xf = np.linspace(0.0, 1.0/(2.0*T), n//2)
  fig, ax = plt.subplots()
  ax.plot(xf, 2.0/n * np.abs(yf[:n//2]))
  plt.grid()
  plt.xlabel("Frequency -->")
  plt.ylabel("Magnitude")
  return plt.show()

def spectrogram(samples, sample_rate, stride_ms = 10.0, window_ms = 20.0, min_freq=2000, max_freq = 10000, eps = 1e-14):
 
  stride_size = int(0.001 * sample_rate * stride_ms)
  window_size = int(0.001 * sample_rate * window_ms)

  # Extract strided windows
  truncate_size = (len(samples) - window_size) % stride_size
  samples = samples[:len(samples) - truncate_size]
  nshape = (window_size, (len(samples) - window_size) // stride_size + 1)
  nstrides = (samples.strides[0], samples.strides[0] * stride_size)
  windows = np.lib.stride_tricks.as_strided(samples, 
                                        shape = nshape, strides = nstrides)
    
  assert np.all(windows[:, 1] == samples[stride_size:(stride_size + window_size)])

  # Window weighting, squared Fast Fourier Transform (fft), scaling
  weighting = np.hanning(window_size)[:, None]
    
  fft = np.fft.rfft(windows * weighting, axis=0)
  fft = np.absolute(fft)
  fft = fft**2
    
  scale = np.sum(weighting**2) * sample_rate
  fft[1:-1, :] *= (2.0 / scale)
  fft[(0, -1), :] /= scale
    
  # Prepare fft frequency list
  freqs = float(sample_rate) / window_size * np.arange(fft.shape[0])
  
  # Compute spectrogram feature
  startind=np.where( freqs >=min_freq)[0][0] 
  endind = np.where( freqs <=max_freq)[0][-1] + 1
  #print(str(startind)+":"+str(endind))
  specgram = np.log(fft[startind:endind, :] + eps)
  return specgram
  
def plot_bird(file_path):
  samples, sampling_rate = librosa.load(file_path, sr= None, mono=True, offset=0.0, duration = None)
  spec=spectrogram(samples,sampling_rate)
  plt.imshow(spec)
  plt.imsave("file_path")
  plt.xlabel(file_path+" Time frame windows")
  plt.ylabel("Frequency")
  plt.show()

def eval_amplitude(samples):
  max=-1
  sum=0.0
  for s in samples:
    #print(s)
    amp=s*s
    if amp > max:
      max=amp
    sum=sum+amp
  return(max,math.sqrt(sum/len(samples)),math.sqrt(sum))

def splice_sound_file(file_path,splice_length,shift):
  samples, sampling_rate = librosa.load(file_path, sr= None, mono=True, offset=0.0, duration = None)
  print("file length ",len(samples))
  sounds=[]
  remaining=len(samples)
  start=0
  ii=0
  #print("splice_length "+str(splice_length))
  #print("shift "+str(shift))
  while(remaining > 0 ):
    #print("remaining: "+str(int(remaining)))
    end=splice_length
    if(remaining < splice_length):
      if(remaining > (splice_length/2)):
        #more than half of time remaining
        #back it up
        #print("greater than half") 
        start -=int((splice_length-remaining)) 
        end=int(splice_length)        
        if start < 0:
          #print("start less than 0")
          #clip is less than splice_length, add empty to end
          start=0
          end=remaining
          sound=samples[start:end]
          add=np.empty((1,splice_length-end))
          add[:]=0.0
          sound=np.append(sound,add)
          sounds.append(sound)
          return sounds
      else:
        return sounds
    #print(str(int(start))+":"+str(int(start+end)))
    sounds.append(samples[int(start):int(start+end)])
    remaining =remaining -shift
    start=start+shift
    ii+=1
  return sounds

def spec_to_file(spec, img_file_path):
  #plt.axis('off')
  #plt.axis('tight')
  np.save(img_file_path+".npy",spec)
  img=plt.imshow(spec)
  plt.savefig(img_file_path+".png")
  #now save data in .txt file
  
#Note: This will parse all subdirectories of path.
def process_training_dir_audio(path,specpath,label,labels,sampling_rate,segment_length, shift, minAmp=0.01):
  print(label)
  images=[] #np.empty((0,161,149))
  test_images=[] #np.empty((0,161,149))
  classes=[]
  test_classes=[]
  display=False
  dirs = os.listdir(path)
  useLabel=True
  dirCount=0
  for entry in dirs:
    file_path=os.path.join(path, entry)
    print(file_path)
    if os.path.isfile(file_path):
      if  ".wav" in file_path:
        
        if useLabel:
          labels.append(label);
          useLabel=False;

          print("label "+str(label))
          #rand=1.0 #always put first of each class to test
        classIndex=int(len(labels)-1)
        testFile=False
        #print("processing file: "+file_path)
        if file_path.find("test") >=0:
          testFile=True

        sounds=splice_sound_file(file_path,segment_length,shift)
        countAmps=0
        #print("Num sounds "+str(len(sounds)))
        for sound in sounds:
          (maxAmp,avgAmp,sumAmp)=eval_amplitude(sound)
          print("Avg amp: "+str(avgAmp)+" sum Amp: "+str(sumAmp))
          if maxAmp >= minAmp:
            #Only use if file reaches minimum amplitude
            countAmps+=1
            dirCount=dirCount+1
            spec=spectrogram(sound,sampling_rate)
            img_file=os.path.join(specpath,label+"_"+str(dirCount))
            if(testFile):
              print("test file "+file_path)
              img_file=img_file+"_test"
              test_images.append(spec)
              test_classes=np.append(test_classes,classIndex)
              #rand=0.1 #reset rand to lower
            else:
              images.append(spec)
              #class is the index of the current label
              classes=np.append(classes, classIndex)
            if(display):
              plt.imshow(spec)
              plt.xlabel(path+" Time frame windows") 
              plt.ylabel("Frequency")
              plt.show()
            spec_to_file(spec,img_file)
            #print(img_file)
            # print(spec.dtype)
            # img=(color.convert_colorspace(spec, 'HSV', 'RGB')*255).astype(np.uint8)
            # print(spec.dtype)
            #skimage.io.imsave(img_file, spec)
            #plt.savefig(img_file)
          
            
        #print("sounds over "+str(minAmp)+" amps: "+str(countAmps)+" out of "+ str(len(sounds)))
    else:
      #if its a dir, then the dir's name is the label
      dirCount=0
      print("Dir: "+entry)
      (images2,classes2,test_images2,test_classes2)=process_training_dir_audio(os.path.join(path, entry),os.path.join(specpath, entry),entry,labels,sampling_rate,segment_length,shift,minAmp)
      #print("Num sounds: "+str(len(images2))+" tests sounds: "+str(len(test_images2)))
      if len(images) == 0:
        images=images2
      else:
        print("images: "+str(len(images)))
        print("images2: "+str(len(images2)))
        images = np.append(images,images2,axis=0)
      if len(test_images) == 0:
        test_images = test_images2
      elif len(test_images2) > 0:
        test_images=np.append(test_images,test_images2,axis=0)
      classes=np.append(classes,[classes2])
      test_classes=np.append(test_classes,test_classes2)
  print("finished directory")
  if(len(test_images) > 0):
    test_images=np.stack(test_images)
    #print(test_images.shape)
  if(len(images) > 0):
    images=np.stack(images)
    #print(images.shape)
  return (images,classes,test_images,test_classes)



def process_dir_audio(path,specpath,label,labels,sampling_rate,segment_length, shift, minAmp=0.01):
  images=[] #np.empty((0,161,149))
  display=False
  dirs = os.listdir(path)
  useLabel=True
  dirCount=0
  for entry in dirs:
    file_path=os.path.join(path, entry)
    print("current file: "+file_path)
    if os.path.isfile(file_path):
      if  ".wav" in file_path or ".WAV" in file_path:
        
        if useLabel:
          labels.append(label);
          useLabel=False;

          print("label "+str(label))

        sounds=splice_sound_file(file_path,segment_length,shift)
        countAmps=0
        print("Num sounds "+str(len(sounds)))
        for sound in sounds:
          (maxAmp,avgAmp,sumAmp)=eval_amplitude(sound)
          #print("Max amp: "+str(maxAmp)+" avg Amp: "+str(avgAmp))
          if maxAmp >= minAmp:
            #Only use if file reaches minimum amplitude
            countAmps+=1
            dirCount=dirCount+1
            spec=spectrogram(sound,sampling_rate)
            img_file=os.path.join(specpath,label+"_"+str(dirCount))
            images.append(spec)
            #class is the index of the current label
            if(display):
              plt.imshow(spec)
              plt.xlabel(path+" Time frame windows") 
              plt.ylabel("Frequency")
              plt.show()
            spec_to_file(spec,img_file)
            # print(spec.dtype)
            # img=(color.convert_colorspace(spec, 'HSV', 'RGB')*255).astype(np.uint8)
            # print(spec.dtype)
            #skimage.io.imsave(img_file, spec)
            #plt.savefig(img_file)
          
            
        #print("sounds over "+str(minAmp)+" amps: "+str(countAmps)+" out of "+ str(len(sounds)))
    else:
      #if its a dir, then the dir's name is the label
      dirCount=0
      images2=process_dir_audio(os.path.join(path, entry),os.path.join(specpath, entry),entry,labels,sampling_rate,segment_length,shift,minAmp)
      #print("Num sounds: "+str(len(images2))+" tests sounds: "+str(len(test_images2)))
      if len(images) == 0:
        images=images2
      else:
       
        images = np.append(images,images2,axis=0)
      
  if(len(images) > 0):
    images=np.stack(images)
    print(images.shape)
  return images

def spec_from_file(spec_file_path):
  spec = np.load(spec_file_path)
  return spec

def process_training_dir_spectrograms(path,label,labels):
  print(label)
  images=[] #np.empty((0,161,149))
  test_images=[] #np.empty((0,161,149))
  classes=[]
  test_classes=[]
  display=False
  dirs = os.listdir(path)
  useLabel=True
  dirCount=0
  testPercent=0.05
  for entry in dirs:
    file_path=os.path.join(path, entry)
    if os.path.isfile(file_path):
      if  ".npy" in file_path:
        if useLabel:
          labels.append(label);
          useLabel=False;
          print("label "+str(label))
          #rand=1.0 #always put first of each class to test
        classIndex=int(len(labels)-1)
        testFile=False
        print("processing file: "+file_path)
        spec=spec_from_file(file_path)
        #if file_path.find("test") >=0:
        #  testFile=True
        if random() < testPercent:
          testFile=True
        if(testFile):
          print("test file "+file_path)
          test_images.append(spec)
          test_classes=np.append(test_classes,classIndex)
          #rand=0.1 #reset rand to lower
        else:
          images.append(spec)
          #class is the index of the current label
          #print(file_path+" "+str(classIndex))
          classes=np.append(classes, classIndex)
            
        #print("sounds over "+str(minAmp)+" amps: "+str(countAmps)+" out of "+ str(len(sounds)))
    else:
      #if its a dir, then the dir's name is the label
      dirCount=0
      (images2,classes2,test_images2,test_classes2)=process_training_dir_spectrograms(os.path.join(path, entry),entry,labels)
      if len(images) == 0:
        images=images2
      else:

        images = np.append(images,images2,axis=0)
      if len(test_images) == 0:
        test_images = test_images2
      elif len(test_images2) > 0:
        test_images=np.append(test_images,test_images2,axis=0)
      classes=np.append(classes,[classes2])
      test_classes=np.append(test_classes,test_classes2)
 
  if(len(test_images) > 0):
    test_images=np.stack(test_images)
    #print(test_images.shape)
  if(len(images) > 0):
    images=np.stack(images)
    print(images.shape)
  return (images,classes,test_images,test_classes)

